In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
data_dir = "/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/CD4_CD8_final_version/"

## check the patients I can use with enough cells in subtypes


In [2]:
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells_annotated.h5ad')

In [3]:
# get adata_CD4EX and adata_CD8EX
adata_CD4EX = adata_T[adata_T.obs['cell_type'] == 'CD4 EX', :]
adata_CD8EX = adata_T[adata_T.obs['cell_type'] == 'CD8 EX', :]

In [4]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD4EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD4EX = set(valid_patients_CD4EX)
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints in CD8EX
cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD8EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD8EX = set(valid_patients_CD8EX)

/tmp/ipykernel_3302300/2168074212.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()
/tmp/ipykernel_3302300/2168074212.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


In [17]:
print(len(valid_patients_CD4EX), len(valid_patients_CD8EX))

16 12


In [7]:
adata_Endo = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_Endo_cells.h5ad')

In [8]:
adata_Endo

AnnData object with n_obs × n_vars = 5621 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'patient_id_colors', 'pca', 'rank_genes_groups', 'timepoint_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata_Endo = adata_Endo

In [ ]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_Endo.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_Endo = cell_counts[(cell_counts['pre'] > 10) & (cell_counts['on'] > 10)].index.tolist()
valid_patients_Endo = set(valid_patients_Endo)

/tmp/ipykernel_3302300/3106465870.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_M1.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


# get the DEGs of filtered patients of CD4EX and B

In [ ]:
Patients_CD4EX_Endo = list(valid_patients_CD4EX & valid_patients_Endo)
adata_CD4EX_1 = adata_CD4EX[adata_CD4EX.obs['patient_id'].isin(Patients_CD4EX_Endo), :]
adata_Endo_1 = adata_Endo[adata_Endo.obs['patient_id'].isin(Patients_CD4EX_Endo), :]

In [16]:
DEG_1 = paird_ttest(adata_CD4EX_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_1 = DEG_1[DEG_1['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [17]:
# remove the DEGs with . in the gene names
DEG_1 = DEG_1[~DEG_1.index.str.contains('\.')]

In [ ]:
DEG_2 = paird_ttest(adata_Endo_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = DEG_2[DEG_2['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [19]:
DEG_2 = DEG_2[~DEG_2.index.str.contains('\.')]

## IVA

In [ ]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_CD4EX_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_Endo_1, sample_id_col='sample_id')

In [ ]:
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_Endo' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')

In [25]:
# export the gene_df to csv
gene_df.to_csv(data_dir + 'gene_df_CD4EX_Endo.csv')

## load in IV result and perform CIT

In [26]:
DEG_pairs = pd.read_excel(data_dir + 'IV_regression_results_with_FisherZ_CD4EX_Endo.xlsx')

In [27]:
DEG_pairs = DEG_pairs.rename(columns = {'gene_T':'g1', 'gene_cell':'g2'})
calculate_gene_correlation(DEG_pairs, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3576502922093007' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2


,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
0,ADGRG1_T,ADGRG1_Endo,0.114402,-1.031163,5.088847e-02,0.134439,7.423737e-02,0.357650
1,ADGRG1_T,ANGPTL4_Endo,0.169536,-2.417672,5.611321e-03,0.179560,1.689745e-02,0.054007
2,ADGRG1_T,AP3S2_Endo,0.159645,-1.370996,3.950778e-02,0.171291,6.123378e-02,-0.190342
3,ADGRG1_T,APOLD1_Endo,0.090553,-5.782743,3.775373e-10,0.116482,1.890996e-07,0.369090
4,ADGRG1_T,ARHGAP30_Endo,0.162794,-3.989856,1.294365e-04,0.173836,1.441077e-03,0.024082
...,...,...,...,...,...,...,...,...
5584,TUBA3D_T,ZBTB22_Endo,0.075272,-0.507720,1.856826e-02,0.104599,3.699751e-02,0.130965
5585,TUBA3D_T,ZNF230_Endo,0.048959,-0.390637,2.789334e-02,0.085915,4.829054e-02,-0.242819
5586,TUBA3D_T,ZNF487_Endo,0.091260,-0.455924,2.839377e-02,0.116930,4.887366e-02,0.114566
5587,TUBA3D_T,ZNF569_Endo,0.053393,-0.422299,2.479207e-02,0.088338,4.456830e-02,-0.219303


In [28]:
# Combine the two conditions using the logical OR operator
DEG_pairs = DEG_pairs[(DEG_pairs['g1vsg2_correlation'] > 0.45) | (DEG_pairs['g1vsg2_correlation'] < -0.45)]
DEG_pairs = DEG_pairs[(DEG_pairs['p_value_ci'] > 0.1)]
# sort the DEG_pairs by g1vsg2_correlation
DEG_pairs = DEG_pairs.sort_values(by='g1vsg2_correlation', ascending=False)
DEG_pairs

,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
4784,TAGAP_T,ARRDC2_Endo,0.000006,0.682927,0.942563,0.003217,0.943914,0.834058
5562,TUBA3D_T,PDK4_Endo,0.000080,0.656442,0.358538,0.012001,0.377093,0.833375
977,CH25H_T,ARRDC2_Endo,0.000030,0.598758,0.105516,0.007905,0.131402,0.821785
5190,TSC22D3_T,ATOH8_Endo,0.000079,0.629045,0.461579,0.012001,0.477425,0.810253
4850,TAGAP_T,TSC22D3_Endo,0.000005,0.610973,0.209945,0.003217,0.232583,0.800943
...,...,...,...,...,...,...,...,...
1574,DTHD1_T,HID1_Endo,0.007671,0.424107,0.608521,0.055684,0.620285,-0.679175
3923,PTPN13_T,HID1_Endo,0.011909,0.341315,0.245735,0.059325,0.267253,-0.717337
3854,PRDM1_T,KLF8_Endo,0.001640,0.511491,0.618252,0.040551,0.629860,-0.734941
776,CD38_T,KLF8_Endo,0.002096,0.482961,0.364701,0.042810,0.382826,-0.739350


In [29]:
lrpair = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv', sep='\t')

In [30]:
# remove the ppi_prediction and ppi_prediction_go
lrpair = lrpair[lrpair['database'] != 'ppi_prediction']
lrpair = lrpair[lrpair['database'] != 'ppi_prediction_go']
lrpair = lrpair[['from', 'to']]
# change from as L and to as R
lrpair.columns = ['L', 'R']
lrpair

,L,R
0,CXCL1,CXCR2
1,CXCL2,CXCR2
2,CXCL3,CXCR2
3,CXCL5,CXCR2
4,PPBP,CXCR2
...,...,...
1387,COL2A1,SDC1
1388,LAMA1,SV2A
1389,COL2A1,GP6
1390,LAMA1,DAG1


In [31]:
# Check expression in adata_1_pseudo
expressed_genes_1 = adata_1_pseudo.var_names[adata_1_pseudo.X.sum(axis=0) > 0].tolist()

# Check expression in adata_2_pseudo
expressed_genes_2 = adata_2_pseudo.var_names[adata_2_pseudo.X.sum(axis=0) > 0].tolist()

# Filter lrpair where both L and R are expressed in their respective datasets
lrpair = lrpair[
    (lrpair['L'].isin(expressed_genes_1)) & 
    (lrpair['R'].isin(expressed_genes_2))
].reset_index(drop=True)
lrpair

,L,R
0,CXCL9,CXCR3
1,CXCL10,CXCR3
2,CXCL11,CXCR3
3,CXCL12,CXCR4
4,CXCL13,CXCR5
...,...,...
211,COL2A1,ITGA11
212,COL2A1,ITGB8
213,COL2A1,CD44
214,COL2A1,SDC1


## CIT Fisherz

In [32]:
results = CIT_test(DEG_pairs, lrpair, adata_1_pseudo, adata_2_pseudo, gene_df, method = 'fisherz', p_value_threshold=0.05)

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib

In [33]:
#calculate the correlation between g1 and g2
calculate_gene_correlation(results, adata_1_pseudo, adata_2_pseudo)
calculate_residuals_correlation(results, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6648869973577287' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2
/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:1060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3674388563485386' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.at[index, 'g1_residualvsg2_residuals_correlation'] = corr


,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,PMAIP1_T,CREB5_Endo,HBEGF,ERBB4,0.070612,0.664887,0.367439
1,PMAIP1_T,CDKN1A_Endo,HBEGF,EGFR,0.078856,0.639174,0.358070
2,CH25H_T,CREB5_Endo,HBEGF,ERBB4,0.118785,0.622734,0.320828
3,PMAIP1_T,ERRFI1_Endo,CCL22,CCR4,0.057292,0.620626,0.384484
4,PMAIP1_T,ERRFI1_Endo,HBEGF,EGFR,0.223016,0.620626,0.254104
...,...,...,...,...,...,...,...
919,BIRC3_T,CXCR3_Endo,CCL5,CXCR3,0.052086,-0.552199,-0.391970
920,BIRC3_T,CXCR3_Endo,CYR61,ITGB2,0.096754,-0.552199,-0.339992
921,SYTL3_T,CRACR2B_Endo,OSM,OSMR,0.058314,-0.564252,-0.383075
922,ERN1_T,HID1_Endo,ITGA4,VCAM1,0.088251,-0.573280,-0.348243


In [34]:
results_filtered = results[(results['pValue'] > 0.1)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] < 0.3)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] > -0.3)]
results_filtered 

,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
4,PMAIP1_T,ERRFI1_Endo,HBEGF,EGFR,0.223016,0.620626,0.254104
5,PMAIP1_T,ERRFI1_Endo,HBEGF,ERBB4,0.306853,0.620626,0.214476
12,CITED2_T,ERRFI1_Endo,HBEGF,EGFR,0.188674,0.604473,0.273141
13,CITED2_T,ERRFI1_Endo,HBEGF,ERBB4,0.174490,0.604473,0.281683
36,ADGRG1_T,FPGT_Endo,OSM,OSMR,0.304138,0.547603,0.215646
...,...,...,...,...,...,...,...
802,ZFP36L1_T,PLA1A_Endo,NUCB2,ERAP1,0.205708,-0.480022,-0.263435
806,CST7_T,CRACR2B_Endo,OSM,OSMR,0.255048,-0.481616,-0.238006
824,CTSL_T,PLA1A_Endo,NUCB2,ERAP1,0.170329,-0.483680,-0.284278
889,F2R_T,H1F0_Endo,SPP1,ITGA4,0.156023,-0.505801,-0.293545


In [35]:
results_filtered['L_R'] = results_filtered['L'] + '_' + results_filtered['R']
results_filtered['L_R'].nunique()

44

In [36]:
results_filtered.to_csv(data_dir + 'CIT_test_results_CD4EX_Endo_results.csv')

In [37]:
import pandas as pd
import numpy as np

# Sample data (assuming results_filtered is already available)
# Adjusting the script to use absolute mean and determine final sign based on majority sign

grouped_results = results_filtered.groupby(["L", "R"])

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Calculate mean correlation values using absolute mean
    g1vsg2_corr_mean = np.abs(group["g1vsg2_correlation"]).mean()
    g1_res_vs_g2_res_mean = np.abs(group["g1_residualvsg2_residuals_correlation"]).mean()
    p_value_mean = group["pValue"].mean()
    
    # Determine final sign based on majority sign of correlations
    sign_g1vsg2 = np.sign(group["g1vsg2_correlation"]).sum()
    sign_g1_res_vs_g2_res = np.sign(group["g1_residualvsg2_residuals_correlation"]).sum()
    
    final_g1vsg2_corr = g1vsg2_corr_mean * (1 if sign_g1vsg2 >= 0 else -1)
    final_g1_res_vs_g2_res = g1_res_vs_g2_res_mean * (1 if sign_g1_res_vs_g2_res >= 0 else -1)
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1
    if len(gem1) > min_gene_num:
        temp_data.append({
            "Ligand": ligand,
            "Receptor": receptor,
            "GEM1_elements": gem1,
            "GEM2_elements": gem2,
            "Num_genes_GEM1": len(gem1),
            "Num_genes_GEM2": len(gem2),
            "GEM1vsGEM2_corr(mean)": final_g1vsg2_corr,
            "GEM1_residualvsGEM2_residual_corr(mean)": final_g1_res_vs_g2_res,
            "average_significance": p_value_mean
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)
grouped_df

,Ligand,Receptor,GEM1_elements,GEM2_elements,Num_genes_GEM1,Num_genes_GEM2,GEM1vsGEM2_corr(mean),GEM1_residualvsGEM2_residual_corr(mean),average_significance
0,APOE,LDLR,"[SFMBT2_T, ADGRG1_T, F2R_T, TNFRSF25_T]","[CXCL9_Endo, WARS_Endo, PLA1A_Endo, CRACR2B_En...",4,6,-0.478342,-0.257404,0.221225
1,CCL19,CCR7,"[NR4A2_T, SLC2A3_T, SH2B3_T, RGS1_T]","[CRACR2B_Endo, SPAAR_Endo]",4,2,0.467438,0.230523,0.284962
2,CCL19,CCRL2,"[NR4A2_T, SLC2A3_T, SH2B3_T, RGS1_T]",[CRACR2B_Endo],4,1,0.469644,0.187615,0.380783
3,CCL5,CCR4,"[NR4A2_T, SLC2A3_T, JUNB_T]",[CRACR2B_Endo],3,1,0.468746,0.277587,0.182066
4,HBEGF,EGFR,"[PMAIP1_T, CITED2_T, CH25H_T, OGFRL1_T, NFKBIA...","[ERRFI1_Endo, CDKN1A_Endo, CAVIN2_Endo, FKBP5_...",7,5,0.512355,0.230314,0.281502
5,HBEGF,ERBB4,"[PMAIP1_T, CITED2_T, CH25H_T, OGFRL1_T, NFKBIA_T]","[ERRFI1_Endo, CDKN1A_Endo, CAVIN2_Endo, FKBP5_...",5,6,0.520492,0.247734,0.252928
6,ITGB1,VCAM1,"[ERN1_T, CST7_T, F2R_T]","[CXCL9_Endo, PLA1A_Endo, WARS_Endo]",3,3,0.497864,0.249685,0.238260
7,LIF,LIFR,"[NR4A2_T, SLC2A3_T, SH2B3_T, RGS1_T]",[CRACR2B_Endo],4,1,0.469644,0.259465,0.217119
8,NUCB2,ERAP1,"[ERN1_T, GPRIN3_T, IL6ST_T, ZFP36L1_T, CTSL_T]","[PLA1A_Endo, WARS_Endo, CX3CL1_Endo]",5,3,0.481801,0.270889,0.194958
9,OSM,OSMR,"[ADGRG1_T, CST7_T, SYTL3_T, SFMBT2_T]","[FPGT_Endo, WARS_Endo, CRACR2B_Endo]",4,3,0.483431,0.204680,0.339832


In [38]:
# rank each GEM_g1 by order of name inside
grouped_df['GEM1_elements'] = grouped_df['GEM1_elements'].apply(lambda x: sorted(x))
grouped_df['GEM2_elements'] = grouped_df['GEM2_elements'].apply(lambda x: sorted(x))

In [39]:
# save the grouped_df to excel
grouped_df.to_excel(data_dir + 'CIT_results_CD4EXvsEndo_groupedLR.xlsx')